In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

In [ ]:
# Load dataset into a pandas dataframe
df = pd.read_excel('./data/kimchi_dataset.xlsx')
df

In [ ]:
# Checking data types in dataset
df.dtypes

In [ ]:
# Verify if XLarge Boxes has any meaningful data
df['XLarge Boxes'].sum()

Verifying the integrity of the "Total Boxes" column, is it really the sum of all the boxes?

In [ ]:
total_boxes = df['Small Boxes'] + df['Large Boxes'] + df['XLarge Boxes']
equals = df['Total Boxes'].equals(total_boxes)
equals

In [ ]:
diff = df['Total Boxes'].compare(total_boxes)
diff

There is no difference. The reason the equals method returns False is probably due to the way the float values are stored which makes them non-identical but essentially the same number.

In [ ]:
# Rename columns to make it easier to access
df = df.rename(columns={"Total Volume": "Volume", "Total Boxes": "Boxes_T", "Small Boxes": "Boxes_S", "Large Boxes": "Boxes_L", "XLarge Boxes": "Boxes_XL"})
df.head()

In [ ]:
# Checking for duplicate rows
duplicate_rows_df = df[df.duplicated()]
print("number of duplicated rows: ", duplicate_rows_df.shape)

In [ ]:
# Checking that number of items match for all columns
df.count()

In [ ]:
# Handling missing values (null or na)
df.isnull().sum()

In [ ]:
# There are 4 missing values for price and 1 for volume. Since 5 is a small portion of the data I will simply drop the rows with missing values.
df = df.dropna()
df.count()

Now that the dataset is clean (no missing values) we can continue with EDA by looking for outliers and skewed data.

In [ ]:
# Plot distribution
plt.figure(figsize=[16, 8])
plt.subplot(2,2,1)
sns.distplot(df['Price'])

plt.subplot(2,2,2)
sns.distplot(df['Volume'])

plt.subplot(2,2,3)
sns.distplot(df['Boxes_S'])

plt.subplot(2,2,4)
sns.distplot(df['Boxes_L'])

plt.show()

In [ ]:
# Detecting outliers using boxplot
plt.figure(figsize=[16, 8])
plt.subplot(2,2,1)
sns.boxplot(x=df['Price'])

plt.subplot(2,2,2)
sns.boxplot(x=df['Volume'])

plt.subplot(2,2,3)
sns.boxplot(x=df['Boxes_S'])

plt.subplot(2,2,4)
sns.boxplot(x=df['Boxes_L'])

In [ ]:
# As you can see in the plots above, the data is quite skewed.
# Try removing some outliers using the Interquartile Range (IQR) technique that is suitable for skewed data.
# sub_df = df[["Price", "Volume", "Boxes_T", "Boxes_S", "Boxes_L", "Boxes_XL"]]
Q1 = df["Price"].quantile(0.25)
Q3 = df["Price"].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Finding lower and upper limits
upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR

In [ ]:
# Finding outliers
df[df['Price'] < lower_limit]

In [ ]:
# There are no outliers lower than the 25 percentile range. Focusing on the upper limit. 
df[df['Price'] > upper_limit]

In [ ]:
# Trimming outliers
trim_df = df[df['Price'] < upper_limit]
trim_df.shape

In [ ]:
# Compare plots after trimming
plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
sns.distplot(x=df['Price'])
plt.subplot(2,2,2)
sns.boxplot(x=df['Price'])
plt.subplot(2,2,3)
sns.distplot(x=trim_df['Price'])
plt.subplot(2,2,4)
sns.boxplot(x=trim_df['Price'])
plt.show()

In [ ]:
# Capping
df_cap = df.copy()
df_cap['Price'] = np.where(
    df_cap['Price'] > upper_limit,
    upper_limit,
    np.where(
        df_cap['Price'] < lower_limit,
        lower_limit,
        df_cap['Price']
    )
)

In [ ]:
df_cap.shape

In [ ]:
# Compare plots after capping
plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
sns.distplot(x=df['Price'])
plt.subplot(2,2,2)
sns.boxplot(x=df['Price'])
plt.subplot(2,2,3)
sns.distplot(x=df_cap['Price'])
plt.subplot(2,2,4)
sns.boxplot(x=df_cap['Price'])
plt.show()

In [ ]:
# Comparing other features
plt.figure(figsize=(16,8))
plt.subplot(2,2,1)
sns.boxplot(x=df_cap['Volume'])
plt.subplot(2,2,2)
sns.boxplot(x=df_cap['Boxes_S'])
plt.subplot(2,2,3)
sns.boxplot(x=df_cap['Boxes_L'])
plt.subplot(2,2,4)
sns.boxplot(x=df_cap['Boxes_XL'])
plt.show()

Even though the other columns are also very skewed (right-skewed), I'll not trim or cap them because doing so will reduce the dataset size that is already quite small. 
There are 2 options here: 
1. Transform data to approximate normal distribution.
2. Use a Standard Scaler and don't fix skew.
2. Use tree based model for regression as they are not affected by skewed data.

For transformation, there are many options
1. log transformation
2. Normalize (min-max)
3. Cube root: used when values are too large. Can be applied to negative values.
4. Square root: Applied only to positive values.
5. Reciprocal
6. Square: apply on left skew (not this case)
7. Box Cox transformation
8. Quantile transform
9. Power transform

In [ ]:
# Let's try Power transform and quantile transform
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
cols1 = ["Price", "Volume", "Boxes_S"]
def test_transformers(columns):
    pt = PowerTransformer()
    qt = QuantileTransformer(n_quantiles=500, output_distribution='normal')
    fig = plt.figure(figsize=(16,24))
    j = 1
    for i in columns:
        array = np.array(df[i]).reshape(-1, 1)
        y = pt.fit_transform(array)
        x = qt.fit_transform(array)
        plt.subplot(3,3,j)
        sns.histplot(array, bins = 50, kde = True)
        plt.title(f"Original Distribution for {i}")
        plt.subplot(3,3,j+1)
        sns.histplot(x, bins = 50, kde = True)
        plt.title(f"Quantile Transform for {i}")
        plt.subplot(3,3,j+2)
        sns.histplot(y, bins = 50, kde = True)
        plt.title(f"Power Transform for {i}")
        j += 3
test_transformers(cols1)

In [ ]:
# Quantile Transform is able to normalize the data nicely so I will use that. Remember that the same preprocessing must be applied to new data later. 
# I'm dropping "Total Boxes" because it won't make sense here.
df = df.drop(["Boxes_T"], axis=1)
df.describe()


In [ ]:
df_trans = df.copy()
cols2 = ["Price", "Volume", "Boxes_S", "Boxes_L", "Boxes_XL"]
def transform_columns(columns):
    qt = QuantileTransformer(n_quantiles=500, output_distribution='normal')
    for i in columns:
        array = np.array(df[i]).reshape(-1, 1)
        df_trans[i] = qt.fit_transform(array)
transform_columns(cols2)
df_trans.describe()

In [ ]:
df_trans.info()

In [ ]:
 # Histogram
plt.figure(figsize=[10, 5])
df_trans['Region'].value_counts().plot(kind="bar", title="Region")

In [ ]:
# Convert date column into separate features
df_trans.Date = pd.to_datetime(df_trans.Date)
df_trans

In [ ]:
df_trans["year"] = df_trans.Date.dt.year
df_trans["month"] = df_trans.Date.dt.month
df_trans["workingday"] = np.where(df_trans.Date.dt.dayofweek < 5, True, False)
df_trans["season"] = df_trans.apply(lambda x: 'winter' if x['month'] in [1,2,12] else ('spring' if x['month'] in [3,4,5] else ('summer' if x['month'] in [6,7,8] else 'autumm')), axis=1)
df_trans

In [ ]:
df_trans.Date.dt.dayofweek.unique()

In [ ]:
# All dates are Sundays and year is 2018, so I'll drop the columns.
df_trans = df_trans.drop(["workingday", "year", "Date"], axis=1)
df_trans

In [ ]:
# Correlation plot
plt.figure(figsize=(10,5))
c = df_trans[["Price", "Volume", "Boxes_S", "Boxes_L", "Boxes_XL","month"]].corr()
sns.heatmap(c, cmap="BrBG", annot=True)
c

In [ ]:
# Correlation between volume and Boxes_S, maybe most of the volume comes from small boxes.
# Price is slightly correlated to Boxes_XL.

In [ ]:
# Checking for correlation between region and price
plt.figure(figsize=(10,5))
sns.barplot(x=df_trans["Region"], y=df_trans["Price"])

# TRAINING

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import tree
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# The dataset has some categorical variables that need to be encoded.
# Nominal: Region
# Ordinal: Season

In [ ]:
# Encoding ordinal variable "season"
df_trans["season"] = df_trans["season"].map({ "winter": 0, "spring": 1, "summer": 2, "autumm": 3})
df_trans

In [ ]:
# Encoding nominal variabe "Region"
one_hot_enc = OneHotEncoder(sparse=False)

In [ ]:
one_hot_enc.fit_transform(df_trans[["Region"]])

In [ ]:
one_hot_enc.categories_

In [ ]:
# Spliting data into training and testing sets
X = df_trans.drop(["Price"], axis=1)
y = df_trans["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
# Make column transformer to make it easier to create scikit-learn pipeline later
# Encoding nominal variabe "Region"
one_hot_enc = OneHotEncoder(sparse=False)
col_trans = make_column_transformer((one_hot_enc, ["Region"]))

In [ ]:
col_trans.fit_transform(X)[:5]

In [ ]:
# Create pipelines from all 3 models

# Instantiate pipeline with linear regression
lm = LinearRegression()
lm_pipeline = make_pipeline(col_trans, lm)

# Instantiate Gradient Boosting Regressor with default loss = 'squared_error' and default criterion = 'friedman_mse'
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(col_trans, gbm)

# Deault criterion for Random Forrest is 'squared_error'
rfm = RandomForestRegressor()
rfm_pipeline = make_pipeline(col_trans, rfm)

In [ ]:
# Train models
lm_pipeline.fit(X_train, y_train)
lm_predictions = lm_pipeline.predict(X_test)
print("First 5 LM predictions: ", list(lm_predictions[:5]))

gbm_pipeline.fit(X_train, y_train)
gbm_predictions = gbm_pipeline.predict(X_test)
print("First 5 GBM predictions: ", list(gbm_predictions[:5]))

rfm_pipeline.fit(X_train, y_train)
rfm_predictions = rfm_pipeline.predict(X_test)
print("First 5 RFM predictions: ", list(rfm_predictions[:5]))

In [ ]:
y_test[:5]

In [ ]:
# Calculate mean absolute error (MAE) and root mean squared error (RMSE)

lm_mae = mean_absolute_error(y_true=y_test, y_pred=lm_predictions)
lm_rmse = np.sqrt(mean_squared_error(y_true=y_test, y_pred=lm_predictions))
print(f"LM MAE: {round(lm_mae, 2)}")
print(f"LM RMSE: {round(lm_rmse, 2)}")

gbm_mae = mean_absolute_error(y_true=y_test, y_pred=gbm_predictions)
gbm_rmse = np.sqrt(mean_squared_error(y_true=y_test, y_pred=gbm_predictions))
print(f"GBM MAE: {round(gbm_mae, 2)}")
print(f"GBM RMSE: {round(gbm_rmse, 2)}")

rfm_mae = mean_absolute_error(y_true=y_test, y_pred=rfm_predictions)
rfm_rmse = np.sqrt(mean_squared_error(y_true=y_test, y_pred=rfm_predictions))
print(f"RFM MAE: {round(rfm_mae, 2)}")
print(f"RFM RMSE: {round(rfm_rmse, 2)}")

## More streamlined approach

In [14]:
# Convert date column into separate features
df.Date = pd.to_datetime(df.Date)
df

,Date,Price,Volume,Boxes_T,Boxes_S,Boxes_L,Boxes_XL,Region
0,2018-03-25,1.71,2321.82,2006.46,1996.46,10.00,0.0,Seoul
1,2018-03-18,1.66,3154.45,2580.60,2577.27,3.33,0.0,Seoul
2,2018-03-11,1.68,2570.52,2209.29,2209.29,0.00,0.0,Seoul
3,2018-03-04,1.48,3851.30,3242.98,3239.65,3.33,0.0,Seoul
4,2018-02-25,1.56,5356.63,4007.48,4007.48,0.00,0.0,Seoul
...,...,...,...,...,...,...,...,...
643,2018-02-04,1.63,17074.83,13498.67,13066.82,431.85,0.0,Boryeong
644,2018-01-28,1.71,13888.04,9264.84,8940.04,324.80,0.0,Boryeong
645,2018-01-21,1.87,13766.76,9394.11,9351.80,42.31,0.0,Boryeong
646,2018-01-14,1.93,16205.22,10969.54,10919.54,50.00,0.0,Boryeong


In [ ]:
df["month"] = df.Date.dt.month
df["season"] = df.apply(lambda x: 0 if x['month'] in [1,2,12] else (1 if x['month'] in [3,4,5] else (2 if x['month'] in [6,7,8] else 4)), axis=1)
df = df.drop(["Date"], axis=1)
df

In [ ]:
# Spliting data into training and testing sets
X = df.drop(["Price", "Boxes_T"], axis=1)
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

In [57]:
def run_model(model_name, pipeline, X_train, y_train, X_test, y_test):
    # Train model
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    mae = mean_absolute_error(y_true=y_test, y_pred=predictions)
    rmse = np.sqrt(mean_squared_error(y_true=y_test, y_pred=predictions))
    print(f"{model_name} MAE: {round(mae, 2)}")
    print(f"{model_name} RMSE: {round(rmse, 2)}")
    return round(rmse, 2)

def run_models(models, X_train, y_train, X_test, y_test):
    mae = []
    rmse = []
    names = []
    for item in models:
        pipeline = item["pipeline"]
        names.append(item["name"])
        pipeline.fit(X_train, y_train)
        predictions = pipeline.predict(X_test)
        mae.append(mean_absolute_error(y_true=y_test, y_pred=predictions))
        rmse.append(np.sqrt(mean_squared_error(y_true=y_test, y_pred=predictions)))
    col={"MAE": mae, "RMSE": rmse}
    ret_df = pd.DataFrame(data=col, index=names)
    return ret_df


### Using StandardScaler

In [53]:
# Make column transformer to make it easier to create scikit-learn pipeline later
# Encoding nominal variabe "Region"
one_hot_enc = OneHotEncoder(sparse=False)
scaler = StandardScaler()
col_trans = make_column_transformer((one_hot_enc, ["Region"]), (scaler, ["Volume", "Boxes_S", "Boxes_L", "Boxes_XL"]))
col_trans.fit_transform(X)[:5]

/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.34245123, -0.33680131, -0.30551989, -0.19348517],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.

In [61]:
# Create pipelines from all models

models = []

# Instantiate pipeline with linear regression
lm = LinearRegression()
lm_pipeline = make_pipeline(col_trans, lm)
# run_model("Linear", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "LinearReg", "pipeline": lm_pipeline})

# Instantiate Gradient Boosting Regressor with default loss = 'squared_error' and default criterion = 'friedman_mse'
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(col_trans, gbm)
# run_model("GB", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "GradBoost", "pipeline": gbm_pipeline})

# Deault criterion for Random Forrest is 'squared_error'
rfm = RandomForestRegressor()
rfm_pipeline = make_pipeline(col_trans, rfm)
# run_model("RandForr", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "RandFor", "pipeline": rfm_pipeline})

# Decision Tree
dtm = tree.DecisionTreeRegressor(max_depth=1)
dtm_pipeline = make_pipeline(col_trans, dtm)
# run_model("DecTree", dtm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "DecTree", "pipeline": dtm_pipeline})

# KNN
knn = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')
knn_pipeline = make_pipeline(col_trans, knn)
# run_model("KNN", knn_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "KNN", "pipeline": knn_pipeline})

# SVM
svm = SVR()
svm_pipeline = make_pipeline(col_trans, svm)
# run_model("SVM", svm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "SVM", "pipeline": svm_pipeline})

results = run_models(models, X_train, y_train, X_test, y_test)
results

/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sk

,MAE,RMSE
LinearReg,10.342081,41.274891
GradBoost,46.800651,373.293766
RandFor,32.443142,227.896476
DecTree,7.421035,45.788963
KNN,5.667163,53.195512
SVM,0.117898,0.160601


### USING QUANTILE TRANSFORMER

In [62]:
# TODO: create pipeline with Transform (normalizer) instead and compare results. Do no normalize price.
from sklearn.preprocessing import QuantileTransformer

In [63]:
# Make column transformer to make it easier to create scikit-learn pipeline later
# Encoding nominal variabe "Region"
one_hot_enc = OneHotEncoder(sparse=False)
quant_trans = QuantileTransformer(n_quantiles=500, output_distribution='normal')
col_trans = make_column_transformer((one_hot_enc, ["Region"]), (quant_trans, ["Volume", "Boxes_S", "Boxes_L", "Boxes_XL"]))
col_trans.fit_transform(X)[:5]

/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -2.50462887, -1.32807964, -0.77070507, -5.19933758],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.

In [64]:
# Create pipelines from all models

models = []

# Instantiate pipeline with linear regression
lm = LinearRegression()
lm_pipeline = make_pipeline(col_trans, lm)
# run_model("Linear", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "LinearReg", "pipeline": lm_pipeline})

# Instantiate Gradient Boosting Regressor with default loss = 'squared_error' and default criterion = 'friedman_mse'
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(col_trans, gbm)
# run_model("GB", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "GradBoost", "pipeline": gbm_pipeline})

# Deault criterion for Random Forrest is 'squared_error'
rfm = RandomForestRegressor()
rfm_pipeline = make_pipeline(col_trans, rfm)
# run_model("RandForr", lm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "RandFor", "pipeline": rfm_pipeline})

# Decision Tree
dtm = tree.DecisionTreeRegressor(max_depth=1)
dtm_pipeline = make_pipeline(col_trans, dtm)
# run_model("DecTree", dtm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "DecTree", "pipeline": dtm_pipeline})

# KNN
knn = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')
knn_pipeline = make_pipeline(col_trans, knn)
# run_model("KNN", knn_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "KNN", "pipeline": knn_pipeline})

# SVM
svm = SVR()
svm_pipeline = make_pipeline(col_trans, svm)
# run_model("SVM", svm_pipeline, X_train, y_train, X_test, y_test)
models.append({"name": "SVM", "pipeline": svm_pipeline})

results = run_models(models, X_train, y_train, X_test, y_test)
results

/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/mattmiranda/Documents/ml-regression/.venv/lib/python3.9/site-packages/sk

,MAE,RMSE
LinearReg,16.655392,43.831407
GradBoost,46.804889,373.293766
RandFor,33.929812,246.927659
DecTree,7.421035,45.788963
KNN,14.366465,91.609868
SVM,0.121549,0.165607
